In [3]:
import pandas as pd
import itertools
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
import nltk
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer  
from nltk.corpus import words
from copy import deepcopy
from gensim.models import Word2Vec
from sklearn import svm

In [4]:
def preproc_text(text):
    text = re.sub(r"\d+", "", text.lower())
    for i in string.punctuation:
        text = text.replace(i,"")
    stop_words = set(stopwords.words("english"))
    tokens = word_tokenize(text)
    sample = [i for i in tokens if not i in stop_words]
    # lancaster_
    stemmer = LancasterStemmer() 
    lemmatizer = WordNetLemmatizer()

    
    
    return [lemmatizer.lemmatize(word) for word in sample]

In [5]:
data = pd.read_csv("movie_data.csv")
data.head()
review = data["review"] # create a data frame with only data 
review = review.apply(preproc_text)

In [7]:
model = Word2Vec(review, size=500, window=10, min_count=10)

In [8]:
def generate_vec(wordlist):
    dic = set(model.wv.index2word)
    feature = np.zeros(500)
    count = 0
    for word in wordlist:
        if word in dic:
            feature = np.add(model.wv[word], feature)
            count += 1
    
    return list(feature/count)
            

In [9]:
def split_train(original_train_data, size=5000):
    return original_train_data[:size], original_train_data[size:size+2000]

def binary_transform(sentiment):
    if sentiment == "positive":
        return 1
    return 0

In [10]:
label = data["sentiment"]
train_X, val_X = split_train(review)
train_y, val_y = split_train(label)
train_y = train_y.apply(binary_transform)
# cleandata = pd.DataFrame({"label":label, "wordlist":review})


In [11]:
train_X = train_X.apply(generate_vec)

In [12]:
X = np.zeros(5000*500).reshape(5000,500)
Y = np.array(train_y)
for i in range(5000):
    X[i] = train_X[i]

In [13]:
clf = svm.SVC(C=2)
clf.fit(X, Y)

SVC(C=2)

In [14]:
VX = val_X.apply(generate_vec)

In [15]:
Vy = val_y.apply(binary_transform)

In [16]:
error = 0
for i in range(2000):
    if clf.predict([VX.values[i]])[0] != Vy.values[i]:
        error += 1
error

260